In [18]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   ---------------------------------------- 5.3/5.3 MB 40.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 79.0 MB/s eta 0:00:35
   ---------------------------------------- 0.0/2.7 GB 80.8 MB/s eta 0:00:34
    --------------------------------------- 0.0/2.7 GB 73.2 MB/s eta 0:00:37
    --------------------------------------- 0.1/2.7 GB 75.8 MB/s eta 0:00:36
   - -------------------------------------- 0.1/2.7 GB 76.2 MB/s eta 0:00:35
   - -------------------------------------- 0.1/2.7 GB 76.8 MB/s eta 0:00:35
   - -------------------------------------- 0.1/2.7 GB 77.1 MB/s eta 0:00:34
   - -------------------------------------- 0.1/2.7 GB 77.4 MB/s eta 0:00:34
   -- ------------------------------------- 0.1/2.7 GB 78.0 MB/s eta 0:00:34
   -- ---------------------------

  You can safely remove it manually.


In [90]:
%pip install rouge-score nltk

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=25026 sha256=421eb17c60a7d060fda23a496fbc781622c2284b665d9e5f2d329b50228dc7cb
  Stored in directory: c:\users\daren\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from pathlib import Path

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define data directories
NOTEBOOK_DIR = Path.cwd()
DATA_DIR = NOTEBOOK_DIR / "data"
INPUTS_DIR = DATA_DIR / "inputs"
OUTPUTS_DIR = DATA_DIR / "outputs"

# text sim

# Ontologie des réseau de transport public du Grand Genève : lignes, véhicules, infrastructures et projets associés

In [ ]:
# === STEP 1: Load the full text from PDF ===
pdf_path = INPUTS_DIR / "Tramway_Geneve_Lignes.pdf"
reader = PdfReader(str(pdf_path))
full_text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

In [116]:
# === STEP 2: Chunk the document into ~512-token chunks ===
def chunk_text(text, max_tokens=512):
    paragraphs = text.split("\n")
    chunks, current_chunk = [], ""
    for para in paragraphs:
        if len((current_chunk + para).split()) < max_tokens:
            current_chunk += " " + para
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

chunks = chunk_text(full_text)

In [117]:
chunks

["Ligne 13 Pour les articles homonymes, voirLigne 13. Laligne 13 du tramway de Genèveest une ancienne ligne diamétrale exploitée par lesTransports publics genevois(TPG) entre1995et2011. Elle desservait, viaCarouge, plusieurs quartiers deGenèveet quelques communes de l'agglomération. Les terminus étaient àNationset aux Palettes, sur la commune deLancy. L'indice est utilisé pour la première fois en1889par la ligne de tramwayGenève-Saint-Julien-en-Genevois, qui fusionne en1925avec laligne 12, puis reprend en1938son indépendance sous la forme d'une ligne d'autobus, laD[1]. La ligne 13 contemporaine est inaugurée le27 mai 1995et est mise en service le lendemain entreGare Cornavinet Bachet-de-Pesay, avec une course sur deux limitée aux Augustins\xa0; elle est ainsi la première ligne de tramway crée àGenèvedepuis la vague de suppression des années 1950 et 60[1]. Elle est aussi la première ligne à inaugurer les Duewag-Vevey DAV à 3 caisses (Be 4/8), issue de la transformation des Be 4/6 à 2 ca

In [119]:
# === STEP 3: Load multilingual BERT for embeddings ===
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

def embed(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**inputs)
    
    # Average the token embeddings (excluding special tokens like [CLS], [SEP] if needed)
    attention_mask = inputs["attention_mask"]
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    sentence_embedding = sum_embeddings / sum_mask
    
    return sentence_embedding[0]


In [120]:
# === STEP 4: Prepare ontology summary ===
ontology_summary = """
Le réseau de transport public de la région s'articule autour d'un ensemble structuré de lignes de tramways, de bus et de trolleybus desservant les principales localités urbaines et périurbaines. Ces lignes, identifiées par leurs numéros ou lettres spécifiques, assurent une connectivité fine entre les différents quartiers, gares, zones industrielles, et points d’intérêt stratégiques comme les institutions, les parcs ou les équipements culturels.

Les lignes de tramway, telles que les lignes 12, 14, ou 17, traversent des zones urbaines denses et desservent des localités emblématiques comme Carouge, Lancy, Meyrin ou encore Nations. Ces lignes sont exploitées par des compagnies de transport comme les TPG et utilisent divers types de véhicules, notamment les Bombardier Cityrunner ou les Siemens Combino, adaptés aux besoins de capacité et aux caractéristiques des voies.

La structuration du territoire est également reflétée dans les zones tarifaires (ex : Zone 10, Zone 210), intégrées dans un système communautaire tel qu’Unireso, qui permet une harmonisation des tarifs à travers différents modes de transport. Le projet Leman Pass s’inscrit également dans cette logique d’intégration tarifaire à l’échelle du Grand Genève.

De nombreux lieux et infrastructures jalonnent le réseau : gares (Cornavin, Lancy-Bachet, Eaux-Vives), stations emblématiques (Place Neuve, Bel-Air), ponts traversant l’Arve ou le Rhône (Pont de la Coulouvrenière, Ponts de l’Île), ou encore routes structurantes (Route de Chêne, Rue de Lausanne). Ces entités sont autant de nœuds stratégiques dans l’organisation de la mobilité.

Certains projets structurants comme le TCMC ou le TCOB visent à étendre ou moderniser le réseau actuel, en lien avec les plans d’urbanisme, les procédures d’enquête publique, ou les concessions ferroviaires nécessaires. D’autres projets, comme les prolongements de lignes ou les travaux préparatoires, sont pensés pour améliorer la desserte de quartiers en développement, tels que les Cherpines ou ZIPLO à Plan-les-Ouates.

Le réseau de transports intègre également des services complémentaires tels que les bus à haut niveau de service ou les lignes nocturnes comme le Noctambus, assurant la continuité de la mobilité au-delà des horaires classiques. Le service est structuré en phases et financé par des acteurs publics, y compris la Confédération suisse.

Enfin, des entités comme le Conseil fédéral, le Grand Conseil du canton de Genève, ou encore des associations telles que l’Association Transports et Environnement participent à la gouvernance de ce système complexe. L’objectif est d’assurer une mobilité fluide, durable, et adaptée aux défis urbains contemporains.

Ce tissu multimodal, maillé autour de lignes, de véhicules, de lieux, et de projets intégrés, constitue un levier stratégique pour la cohésion territoriale, le développement économique, et la transition écologique de la région genevoise.
"""

summary_embedding = embed(ontology_summary)

In [121]:
# === STEP 5: Compute cosine similarities and aggregate ===
def cosine_similarity(a, b):
    return torch.nn.functional.cosine_similarity(a, b, dim=0).item()

similarities = [cosine_similarity(embed(chunk), summary_embedding) for chunk in chunks]

# Aggregate
print(f"\nAverage similarity: {np.mean(similarities):.3f}")
print(f"Max similarity: {np.max(similarities):.3f}")
print(f"Min similarity: {np.min(similarities):.3f}")



Average similarity: 0.935
Max similarity: 0.955
Min similarity: 0.896


In [122]:
# === STEP 6: Compute ROUGE and BLEU between ontology_summary and each chunk ===

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smooth = SmoothingFunction().method4

rouge1_scores, rouge2_scores, rougeL_scores, bleu_scores = [], [], [], []

for chunk in chunks:
    rouge_scores = rouge_scorer.score(ontology_summary, chunk)
    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)
    
    reference = [ontology_summary.split()]
    hypothesis = chunk.split()
    bleu = sentence_bleu(reference, hypothesis, smoothing_function=smooth)
    bleu_scores.append(bleu)

# === STEP 7: Display averaged metrics ===

print(f"\n--- ROUGE & BLEU scores (avg over {len(chunks)} chunks) ---")
print(f"ROUGE-1: {np.mean(rouge1_scores):.3f}")
print(f"ROUGE-2: {np.mean(rouge2_scores):.3f}")
print(f"ROUGE-L: {np.mean(rougeL_scores):.3f}")
print(f"BLEU:    {np.mean(bleu_scores):.3f}")



--- ROUGE & BLEU scores (avg over 13 chunks) ---
ROUGE-1: 0.365
ROUGE-2: 0.059
ROUGE-L: 0.139
BLEU:    0.012


# Ontologie de l’organisation, le développement, l’infrastructure, la planification et les politiques liées aux transports publics

In [ ]:
# === STEP 1: Load the full text from PDF ===
pdf_path = INPUTS_DIR / "loi_réseau_transports_publics.pdf"
reader = PdfReader(str(pdf_path))
full_text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

In [140]:
# === STEP 2: Chunk the document into ~512-token chunks ===
def chunk_text(text, max_tokens=512):
    paragraphs = text.split("\n")
    chunks, current_chunk = [], ""
    for para in paragraphs:
        if len((current_chunk + para).split()) < max_tokens:
            current_chunk += " " + para
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

chunks = chunk_text(full_text)

In [141]:
chunks

['Loi modifiant la loi sur le réseau  des transports publics (LRTP)  (12553)  H 1 50  du 26 juin 2020  Le GRAND CONSEIL de la République et canton de Genève  décrète ce qui suit :  Art. 1  Modifications  La loi sur le réseau des transports publics, du 17 mars 1988 (LRTP – H 1 50),  est modifiée comme suit :  Art. 4, al. 1, phrase introductive, lettres a, b, chiffres 2 et 3, lettres a, c et  d (nouvelle teneur), lettre f (nouvelle), chiffre 4 (nouvelle  teneur), al. 1, lettre c (nouvelle teneur), al. 1, lettre f  (nouvelle), al. 2 (nouvelle teneur)  1 Le réseau des transports publics est renforcé à l’horizon 2030, en  conformité avec les différentes générations du projet d’agglomération et le  plan directeur cantonal, dans le but d’améliorer la desserte urbaine, régionale  et  transfrontalière  de  l’agglomération  et  de  façon  à  augmenter  significativement la capacité d’accueil aux heures de pointe, ceci par les  mesures suivantes :  a) Transports régionaux La desserte régionale et

In [136]:
# === STEP 3: Load multilingual BERT for embeddings ===
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

def embed(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**inputs)
    
    # Average the token embeddings (excluding special tokens like [CLS], [SEP] if needed)
    attention_mask = inputs["attention_mask"]
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    sentence_embedding = sum_embeddings / sum_mask
    
    return sentence_embedding[0]

In [142]:
# === STEP 4: Prepare ontology summary ===
ontology_summary = """
Dans le cadre d’une politique globale de mobilité durable, la région a entrepris une refonte ambitieuse de ses infrastructures de transport. L’objectif principal est d’assurer une desserte efficace et multimodale du territoire, tout en répondant aux enjeux climatiques et aux besoins de mobilité des usagers.

Le système de transport repose sur un maillage équilibré entre transports collectifs, mobilités douces et infrastructures routières. Le tramway, l’autobus et le trolleybus constituent l’épine dorsale du réseau de transports publics, avec des extensions prévues vers les zones périurbaines et transfrontalières. Ces services sont complétés par des autobus électriques à recharge rapide, contribuant à la décarbonation progressive du parc roulant.

Plusieurs projets d’agglomération sont en cours, intégrant des lignes de tramway et des bus à haut niveau de service, aménagés sur des sites propres ou réservés. Le prolongement de certaines lignes et la création de liaisons diamétrales visent à améliorer la connectivité entre les différents pôles urbains. Ces projets s’inscrivent dans une vision à long terme, articulée autour de plans directeurs cantonaux et de programmes de développement stratégique ferroviaire.

Parallèlement, une attention particulière est portée à la capacité d’accueil pendant les heures de pointe, avec la création de gares souterraines, de terminus intermédiaires, et de garages pour le matériel roulant. Le réseau express régional ferroviaire est renforcé, notamment grâce à des projets comme Leman 2030 et la Diamétrale Bernex-Cornavin-Nations-Meyrin-Zimeysa.

La mobilité douce est fortement encouragée à travers le développement de voies vertes, de liaisons cyclables et piétonnes, et de systèmes de vélos en libre-service. Des aménagements cyclables structurants, comme les axes forts vélos, sont en cours de réalisation pour favoriser les déplacements alternatifs à la voiture.

Le stationnement fait également l’objet d’une planification fine : des parkings relais (P+R), des espaces de stationnement pour pendulaires, et des dispositifs de tarification différenciée (gratuits ou payants, selon l’usage et la zone) sont déployés pour accompagner le report modal.

Une gouvernance adaptée soutient l’ensemble de ces initiatives. Le Conseil d’État, les autorités compétentes et les acteurs locaux assurent la coordination des projets via des consultations publiques, des cahiers des charges, et un cadre juridique comprenant lois, concessions, et régulations tarifaires.

Enfin, un volet d’accompagnement du changement est intégré à chaque étape : sensibilisation, formation, communication de proximité, et transformation des comportements sont essentiels pour faire évoluer la culture de la mobilité. Le développement de MaaS (Mobility as a Service), la promotion de la mobilité partagée, et l’intégration du numérique dans la réduction des déplacements sont des leviers d’action majeurs.

L’ensemble de ces actions vise à construire un système de mobilité cohérent, résilient et inclusif à l’horizon 2050, capable de répondre aux défis de la transition énergétique, de la densification urbaine et de la justice spatiale.
"""

summary_embedding = embed(ontology_summary)

In [129]:
# === STEP 5: Compute cosine similarities and aggregate ===
def cosine_similarity(a, b):
    return torch.nn.functional.cosine_similarity(a, b, dim=0).item()

similarities = [cosine_similarity(embed(chunk), summary_embedding) for chunk in chunks]

# Aggregate
print(f"\nAverage similarity: {np.mean(similarities):.3f}")
print(f"Max similarity: {np.max(similarities):.3f}")
print(f"Min similarity: {np.min(similarities):.3f}")


Average similarity: 0.926
Max similarity: 0.962
Min similarity: 0.794


In [145]:
# === STEP 6: Compute ROUGE and BLEU between ontology_summary and each chunk ===

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smooth = SmoothingFunction().method4

rouge1_scores, rouge2_scores, rougeL_scores, bleu_scores = [], [], [], []

for chunk in chunks:
    rouge_scores = rouge_scorer.score(ontology_summary, chunk)
    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)
    
    reference = [ontology_summary.split()]
    hypothesis = chunk.split()
    bleu = sentence_bleu(reference, hypothesis, smoothing_function=smooth)
    bleu_scores.append(bleu)

# === STEP 7: Display averaged metrics ===

print(f"\n--- ROUGE & BLEU scores (avg over {len(chunks)} chunks) ---")
print(f"ROUGE-1: {np.mean(rouge1_scores):.3f}")
print(f"ROUGE-2: {np.mean(rouge2_scores):.3f}")
print(f"ROUGE-L: {np.mean(rougeL_scores):.3f}")
print(f"BLEU:    {np.mean(bleu_scores):.3f}")



--- ROUGE & BLEU scores (avg over 8 chunks) ---
ROUGE-1: 0.392
ROUGE-2: 0.089
ROUGE-L: 0.147
BLEU:    0.018
